In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write
import numpy as np


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

c:\Users\63537\anaconda3\envs\emotional-vits\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hps = utils.get_hparams_from_file("./configs/yua.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("logs/yua/G_8600.pth", net_g, None)

INFO:root:Loaded checkpoint 'logs/yua/G_8600.pth' (iteration 6)


In [14]:
from pathlib import Path
import random
random_emotion_root = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts'
# [i for i in Path(random_emotion_root).rglob('*') if i.suffix=='.wav']
random.sample([i for i in Path(random_emotion_root).rglob('*') if i.suffix == '.wav'],1)

[WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/cuts/06/6-409797-413856.wav')]

In [23]:
import librosa
librosa.load(Path(r"C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts\06\6-10707-12957.wav"), sr=16000)

(array([-7.8469864e-05, -1.4151074e-04, -8.7411943e-05, ...,
        -3.6822809e-04, -3.4117635e-04, -2.1806915e-04], dtype=float32),
 16000)

In [24]:
# 随机抽取情感参考音频的根目录
random_emotion_root = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts'
import random
from pathlib import Path

def tts(txt, emotion:str):
    """emotion为参考情感音频路径 或random_sample（随机抽取）"""
    stn_tst = get_text(txt, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        sid = torch.LongTensor([0])
        # 如果输入是一个emo文件路径
        if Path(emotion).is_file() and Path(emotion).suffix == '.npy':
            emo = torch.FloatTensor(np.load(Path(emotion))).unsqueeze(0)
        # 如果输入是某个目录，那么在目录下随机选择.emo.npy文件
        elif Path(emotion).is_dir():
            rand_emo:Path = random.sample([i for i in Path(random_emotion_root).rglob('*') if i.suffix == '.npy'],1)[0]
            emo = torch.FloatTensor(np.load(Path(rand_emo))).unsqueeze(0)
            print(f"Random emotion: {rand_emo}")
        # 如果输入是某个 wav文件，那么提取 emotion
        elif Path(emotion).is_file() and Path(emotion).suffix == '.wav':
            import emotion_extract
            emo = torch.FloatTensor(emotion_extract.extract_wav(Path(emotion)))
        else:
            raise RuntimeError(f"Wrong path of emotion:{emotion}")

        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=0.667, noise_scale_w=0.8, length_scale=1, emo=emo)[0][0,0].data.float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [28]:
txt = "晚上好晚上好"
tts(txt, emotion=random_emotion_root)

Random emotion: C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts\06\6-597719-600691.wav.emo.npy


In [11]:
# 随机选取使用训练数据中某一条数据的情感
# 随机抽取的音频文件路径可以用于使用该情感合成其他句子
txt = "晚上好晚上好"
tts(txt, emotion=random_emotion_root)
tts(txt, emotion=random_emotion_root)
tts(txt, emotion=random_emotion_root)
tts(txt, emotion=random_emotion_root)
tts(txt, emotion=random_emotion_root)
tts(txt, emotion=random_emotion_root)

dataset/nene/nen108_168.wav


dataset/nene/nen106_128.wav


dataset/nene/nen113_136.wav


dataset/nene/nen405_025.wav


dataset/nene/nen116_030.wav


dataset/nene/nen104_275.wav


In [16]:
txt = "こんにちは。私わあやちねねです。"
tts(txt, emotion="dataset/nene/nen116_030.wav")